In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# =======

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-01-etl-with-nvtabular/nvidia_logo.png" style="width: 90px; float: right;">

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create sequential features with [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular). Such data will be used in the next notebook to train a session-based recommendation model.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Import required libraries

In [2]:
import os
import glob

import numpy as np
import pandas as pd

import cudf
import cupy as cp
import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Define Input/Output Path

In [3]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data/")

## Create a Synthetic Input Data

In [4]:
NUM_ROWS = os.environ.get("NUM_ROWS", 100000)

In [5]:
long_tailed_item_distribution = np.clip(np.random.lognormal(3., 1., int(NUM_ROWS)).astype(np.int32), 1, 50000)
# generate random item interaction features 
df = pd.DataFrame(np.random.randint(70000, 90000, int(NUM_ROWS)), columns=['session_id'])
df['item_id'] = long_tailed_item_distribution

# generate category mapping for each item-id
df['category'] = pd.cut(df['item_id'], bins=334, labels=np.arange(1, 335)).astype(np.int32)
df['age_days'] = np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)
df['weekday_sin']= np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)

# generate day mapping for each session 
map_day = dict(zip(df.session_id.unique(), np.random.randint(1, 10, size=(df.session_id.nunique()))))
df['day'] =  df.session_id.map(map_day)

Visualize couple of rows of the synthetic dataset:

In [6]:
df.head()

,session_id,item_id,category,age_days,weekday_sin,day
0,71434,10,3,0.840850,0.687689,5
1,71597,37,9,0.711453,0.964688,7
2,83491,49,12,0.056856,0.031751,4
3,72565,14,4,0.300812,0.545527,9
4,82352,9,2,0.680995,0.420036,7


## Feature Engineering with NVTabular

Deep Learning models require dense input features. Categorical features are sparse, and need to be represented by dense embeddings in the model. To allow for that, categorical features first need to be encoded as contiguous integers `(0, ..., |C|)`, where `|C|` is the feature cardinality (number of unique values), so that their embeddings can be efficiently stored in embedding layers.  We will use NVTabular to preprocess the categorical features, so that all categorical columns are encoded as contiguous integers. Note that the `Categorify` op encodes OOVs or nulls to `0` automatically. In our synthetic dataset we do not have any nulls. On the other hand `0` is also used for padding the sequences in input block, therefore, you can set `start_index=1` arg in the Categorify op if you want the encoded null or OOV values to start from `1` instead of `0` because we reserve `0` for padding the sequence features.

Here our goal is to create sequential features.  In this cell, we are creating temporal features and grouping them together at the session level, sorting the interactions by time. Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features on GPU with a few lines.

In [7]:
SESSIONS_MAX_LENGTH =20

# Categorify categorical features
categ_feats = ['session_id', 'item_id', 'category'] >> nvt.ops.Categorify()

# Define Groupby Workflow
groupby_feats = categ_feats + ['day', 'age_days', 'weekday_sin']

# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "age_days": ["list"],
        'weekday_sin': ["list"],
        },
    name_sep="-")

# Select and truncate the sequential features
sequence_features_truncated = (
    groupby_features['category-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.ValueCount()
)

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
    >> nvt.ops.ValueCount()
)  
sequence_features_truncated_cont = (
    groupby_features['age_days-list', 'weekday_sin-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
    >> nvt.ops.ValueCount()
)

# Filter out sessions with length 1 (not valid for next-item prediction training and evaluation)
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'day-first', 'session_id'] + 
    sequence_features_truncated_item +
    sequence_features_truncated + 
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['item_id-list', 'category-list', 'age_days-list', 'weekday_sin-list'] >>  nvt.ops.ValueCount()


workflow = nvt.Workflow(filtered_sessions['session_id', 'day-first', 'item_id-count'] + seq_feats_list)

dataset = nvt.Dataset(df, cpu=False)
# Generate statistics for the features
workflow.fit(dataset)
# Apply the preprocessing and return an NVTabular dataset
sessions_ds = workflow.transform(dataset)
# Convert the NVTabular dataset to a Dask cuDF dataframe (`to_ddf()`) and then to cuDF dataframe (`.compute()`)
sessions_gdf = sessions_ds.to_ddf().compute()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [8]:
sessions_gdf.head(3)

,session_id,day-first,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,1,1,16,"[25, 69, 1, 5, 25, 55, 7, 9, 26, 35, 26, 1, 95...","[7, 17, 1, 1, 7, 14, 3, 3, 7, 9, 7, 1, 24, 5, ...","[0.55352765, 0.7910041, 0.941412, 0.51870626, ...","[0.5049706, 0.66169703, 0.34624058, 0.2268689,..."
1,2,2,15,"[4, 22, 1, 21, 24, 16, 29, 9, 9, 8, 52, 123, 1...","[1, 6, 1, 6, 6, 5, 8, 3, 3, 2, 13, 30, 5, 8, 4]","[0.20037235, 0.74481636, 0.6761332, 0.56571895...","[0.26202255, 0.08678413, 0.17604615, 0.8410048..."
2,3,9,15,"[52, 28, 19, 31, 9, 32, 170, 4, 23, 12, 70, 5,...","[13, 7, 2, 2, 3, 8, 44, 1, 6, 2, 18, 1, 1, 2, 1]","[0.6514846, 0.6066424, 0.3734063, 0.92335576, ...","[0.6934628, 0.2280334, 0.77632314, 0.7626327, ..."


It is possible to save the preprocessing workflow. That is useful to apply the same preprocessing to other data (with the same schema) and also to deploy the session-based recommendation pipeline to Triton Inference Server.

In [9]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(Tags.CATEGORICAL),int64,False,False,NaN,0.0,0.0,0.0,.//categories/unique.session_id.parquet,0.0,19860.0,session_id,19861.0,408.0,NaN,NaN
1,day-first,(),int64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_id-count,(Tags.CATEGORICAL),int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.item_id.parquet,0.0,490.0,item_id,491.0,51.0,NaN,NaN
3,item_id-list,"(Tags.ITEM, Tags.LIST, Tags.CATEGORICAL, Tags....",int64,True,True,NaN,0.0,0.0,0.0,.//categories/unique.item_id.parquet,0.0,490.0,item_id,491.0,51.0,2.0,16.0
4,category-list,"(Tags.LIST, Tags.CATEGORICAL)",int64,True,True,NaN,0.0,0.0,0.0,.//categories/unique.category.parquet,0.0,173.0,category,174.0,29.0,2.0,16.0
5,age_days-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,16.0
6,weekday_sin-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,16.0


The following will generate `schema.pbtxt` file in the provided folder.

In [10]:
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt"))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [11]:
workflow.save(os.path.join(INPUT_DATA_DIR, "workflow_etl"))

## Export pre-processed data by day

In this example we are going to split the preprocessed parquet files by days, to allow for temporal training and evaluation. There will be a folder for each day and three parquet files within each day folder: `train.parquet`, `validation.parquet` and `test.parquet`.

In [12]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR",os.path.join(INPUT_DATA_DIR, "sessions_by_day"))

In [13]:
from transformers4rec.data.preprocessing import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_DIR,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                      )

Creating time-based splits: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 22.81it/s]


## Checking the preprocessed outputs

In [14]:
TRAIN_PATHS = os.path.join(OUTPUT_DIR, "1", "train.parquet")

In [15]:
gdf = cudf.read_parquet(TRAIN_PATHS)
gdf

,session_id,item_id-count,item_id-list,category-list,age_days-list,weekday_sin-list
0,1,16,"[25, 69, 1, 5, 25, 55, 7, 9, 26, 35, 26, 1, 95...","[7, 17, 1, 1, 7, 14, 3, 3, 7, 9, 7, 1, 24, 5, ...","[0.55352765, 0.7910041, 0.941412, 0.51870626, ...","[0.5049706, 0.66169703, 0.34624058, 0.2268689,..."
1,25,13,"[4, 3, 32, 39, 18, 51, 29, 7, 1, 5, 66, 9, 48]","[1, 2, 8, 10, 5, 13, 8, 3, 1, 1, 17, 3, 12]","[0.5546555, 0.27420917, 0.8775895, 0.9767382, ...","[0.11418641, 0.93307984, 0.20312999, 0.6421522..."
2,26,13,"[14, 11, 6, 25, 13, 27, 19, 221, 29, 4, 11, 7,...","[4, 4, 3, 7, 4, 7, 2, 56, 8, 1, 4, 3, 4]","[0.27867177, 0.17603067, 0.77485156, 0.5364172...","[0.6755759, 0.18836002, 0.13808286, 0.3261594,..."
4,43,12,"[3, 20, 22, 11, 22, 6, 9, 7, 4, 17, 16, 60]","[2, 5, 6, 4, 6, 3, 3, 3, 1, 5, 5, 15]","[0.30618587, 0.89246064, 0.76363343, 0.4738950...","[0.7805741, 0.995789, 0.42185915, 0.13173895, ..."
5,45,12,"[1, 7, 17, 16, 14, 11, 14, 14, 50, 11, 4, 88]","[1, 3, 5, 5, 4, 4, 4, 4, 13, 4, 1, 22]","[0.23821828, 0.9370317, 0.74576575, 0.10352959...","[0.6149876, 0.6672345, 0.50857466, 0.5500414, ..."
...,...,...,...,...,...,...
2079,19155,2,"[22, 65]","[6, 16]","[0.09291915, 0.81337434]","[0.608028, 0.19263099]"
2080,19160,2,"[70, 20]","[18, 5]","[0.7828527, 0.60427713]","[0.7843421, 0.04484622]"
2081,19181,2,"[9, 11]","[3, 4]","[0.7982899, 0.35936186]","[0.40564352, 0.65942895]"
2082,19201,2,"[29, 14]","[8, 4]","[0.26832914, 0.6700629]","[0.21050844, 0.45085543]"


You have  just created session-level features to train a session-based recommendation model using NVTabular. Now you can move to the the next notebook,`02-session-based-XLNet-with-PyT.ipynb` to train a session-based recommendation model using [XLNet](https://arxiv.org/abs/1906.08237), one of the state-of-the-art NLP model. Please shut down this kernel to free the GPU memory before you start the next one.